In [1]:
import pandas as pd

df = pd.read_csv('./encoded-train.csv')
df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Kmｲ),Density (P/Kmｲ)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [2]:
data = {'text': df['selected_text'], 'sentiment': df['sentiment']}
df = pd.DataFrame(data)
df.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27481 non-null  object
dtypes: object(2)
memory usage: 429.5+ KB


In [4]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27480 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27480 non-null  object
dtypes: object(2)
memory usage: 644.1+ KB


In [5]:
df['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [6]:
df1 = df[df['text'].str.contains(r'^[a-zA-Z\s]*$')]
print(df1.head())

             text sentiment
1        Sooo SAD  negative
2     bullying me  negative
3  leave me alone  negative
6             fun  positive
7      Soooo high   neutral


In [7]:
from sklearn.model_selection import train_test_split

X = df1['text']
y = df1['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\svmra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
tokenizer = RegexpTokenizer(r"\w+")
en_stopwords = set(stopwords.words('english'))
ps = PorterStemmer()

In [10]:
def getCleanedText(text):
  text = text.lower()

  # tokenizing
  tokens = tokenizer.tokenize(text)
  new_tokens = [token for token in tokens if token not in en_stopwords]
  stemmed_tokens = [ps.stem(tokens) for tokens in new_tokens]
  clean_text = " ".join(stemmed_tokens)
  return clean_text

In [11]:
X_clean = [getCleanedText(i) for i in X_train]
xt_clean = [getCleanedText(i) for i in X_test]

In [12]:
X_clean[:10]

['cool',
 'enjoy',
 'thank',
 'cost',
 'hyster',
 'happi',
 'document day',
 'broke',
 'im happi',
 'unfortun']

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range = (1,2))
X_vec = cv.fit_transform(X_clean).toarray()

X_vec

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
print(cv.get_feature_names_out())

['aaaaaaaaaaa' 'aaaaaaaaaaa mcfli' 'aaaaaw' ... 'zoo today' 'zu'
 'zu pinkpop']


In [15]:
Xt_vect = cv.transform(xt_clean).toarray()
Xt_vect

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
from sklearn.naive_bayes import MultinomialNB
mn = MultinomialNB()
mn.fit(X_vec, y_train)

MultinomialNB()

In [17]:
y_pred = mn.predict(Xt_vect)
y_pred

array(['negative', 'positive', 'positive', ..., 'positive', 'neutral',
       'positive'], dtype='<U8')

In [18]:
from sklearn.metrics import accuracy_score


accuracy = accuracy_score(y_test, y_pred)
print(accuracy*100)

82.17775709632387


In [19]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.88      0.78      0.83       821
     neutral       0.75      0.54      0.63       356
    positive       0.80      0.96      0.87       972

    accuracy                           0.82      2149
   macro avg       0.81      0.76      0.78      2149
weighted avg       0.82      0.82      0.81      2149



In [20]:
import pickle

pickle.dump(mn,open('final.pkl','wb'))